In [87]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()

db = client.restaurants

for rest in db.restaurants.find({"name":"Burger King"}):
    print("BK placed in {} {}".format(rest["borough"], rest["address"]["street"]))


BK placed in Queens Northern Boulevard
BK placed in Queens Northern Boulevard
BK placed in Queens Northern Boulevard
BK placed in Queens Hillside Avenue
BK placed in Brooklyn Church Avenue
BK placed in Queens Queens Boulevard
BK placed in Queens Rockaway Boulevard
BK placed in Queens 61 Avenue
BK placed in Staten Island Jewett Avenue
BK placed in Queens 82 Street
BK placed in Queens Astoria Boulevard
BK placed in Bronx White Plains Road
BK placed in Manhattan Beaver Street
BK placed in Staten Island Hylan Boulevard
BK placed in Staten Island Willow Brook Road
BK placed in Staten Island Hylan Boulevard
BK placed in Staten Island Richmond Avenue
BK placed in Brooklyn South Conduit Avenue
BK placed in Brooklyn Marcy Avenue
BK placed in Manhattan West  125 Street
BK placed in Queens Main Street
BK placed in Queens Merrick Boulevard
BK placed in Brooklyn Hillel Place
BK placed in Bronx East  149 Street
BK placed in Queens Atlantic Avenue
BK placed in Brooklyn 4 Avenue
BK placed in Brooklyn 

In [91]:
from pandas.io.json import json_normalize

def prepareForTableau(bks):
    locations = json_normalize(bks["address"])
    gelocs = locations.apply(lambda e: [e["coord"][0],e["coord"][1]], result_type="expand", axis=1)
    clean_df = pd.concat([bks["restaurant_id"],locations[["street","zipcode"]],gelocs], axis=1)
    clean_df.rename({1:"lat",0:"long"}, axis=1, inplace=True)
    return clean_df

In [92]:
bks = pd.DataFrame(db.restaurants.find({"name":"Burger King"}))
bks.head()

clean_df = prepareForTableau(bks)
display(clean_df.head())


,restaurant_id,street,zipcode,long,lat
0,40370167,Northern Boulevard,11361,-73.759249,40.761574
1,40370238,Northern Boulevard,11377,-73.897071,40.754390
2,40370239,Northern Boulevard,11354,-73.820668,40.764744
3,40370916,Hillside Avenue,11004,-73.700739,40.738955
4,40370917,Church Avenue,11236,-73.919511,40.652764


In [90]:
clean_df.to_json("bk_rests.json",orient="records")

In [102]:

nearLocation = {
    "lng": 40.717432,
    "lat": -73.994943
}

bks = pd.DataFrame(db.restaurants.find({
    "address.coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 2000, # In meters
     }
   }
}))
bks.head()



OperationFailure: error processing query: ns=restaurants.restaurantsTree: GEONEAR  field=address.coord maxdist=2000 isNearSphere=0
Sort: {}
Proj: {}
 planner returned error: unable to find index for $geoNear query

In [101]:
clean_df = prepareForTableau(bks)
display(clean_df.head())
clean_df.to_json("bk_rests_filtered.json",orient="records")

,restaurant_id,street,zipcode,long,lat
0,50011542,Bowery,10002,-73.995023,40.717325
1,41696848,Hester Street,10002,-73.994962,40.717214
2,41137336,Chrystie Street,10002,-73.994584,40.717339
3,41615445,Chrystie Street,10002,-73.994584,40.717339
4,41537020,Chrystie Street,10002,-73.994526,40.717451
